Lectura y expansión de la señal x[n]

In [3]:
import numpy as np
import soundfile as sf
from scipy.fft import dct, idct


def leer_y_expandir(wavfile, N=64):
    """
    Lee el WAV, retorna (xE, fs).
    xE es la señal expandida, su longitud es múltiplo de N.
    """
    x, fs = sf.read(wavfile)  # x en range [-1,1] si es float
    # Si es estéreo, tomar un canal o mezclar (a elección):
    if x.ndim > 1:
        x = x[:, 0]  # Tomamos canal izquierdo, por ejemplo

    L = len(x)
    # Calculamos número de ceros para completar múltiplo de N:
    resto = L % N
    if resto == 0:
        xE = x
    else:
        ceros = np.zeros(N - resto, dtype=x.dtype)
        xE = np.concatenate((x, ceros))

    return xE, fs

Cálculo vectorizado de DCT por bloques

In [4]:
def dct_bloques(xE, N=64):
    """
    xE es un array 1D de longitud multiple de N.
    Retorna una matriz C de forma (N, Nb),
    donde Nb = numero de bloques = len(xE)/N,
    y C[:,b] es la DCT( xE[b*N : b*N+N] ).
    """
    # Reformar
    xE_2d = xE.reshape(-1, N).T  # Tamaño (N, Nb) si reshape(-1,N)
    # Calculamos la DCT a lo largo de la dimensión 0 (cada columna un bloque)
    # Por convención, la DCT de scipy actúa a lo largo del eje=0 si no se especifica:
    #   dct(x, axis=0) => transformada por filas,
    # pero ahora nuestras "filas" son en realidad muestras del bloque,
    # y nuestras "columnas" son distintos bloques.
    C = dct(xE_2d, axis=0, norm="ortho")
    return C

In [5]:
def idct_bloques(C):
    """
    Recibe la matriz C (N, Nb),
    donde cada columna es la DCT de un bloque de N muestras.
    Retorna un vector yE con la reconstrucción por IDCT (concatenada).
    """
    # IDCT por columnas
    X_2d = idct(C, axis=0, norm="ortho")
    # X_2d es (N, Nb). Lo transponemos e "aplanamos"
    X_1d = X_2d.T.reshape(-1)
    return X_1d

Cálculo de la compacción (ganancia G)

In [6]:
# X_blocks es (N, Nb):
#  la fila i corresponde a "todas las realizaciones de la variable aleatoria X_i"
vars_X = np.var(X_blocks, axis=1, ddof=1)  # ddof=1 => var muestral

vars_C = np.var(C_blocks, axis=1, ddof=1)


def ganancia_compaccion(vars_array):
    # media aritmética
    ma = np.mean(vars_array)
    # media geométrica
    # Evitamos problemas con log(0).
    # Podríamos ignorar coeficientes con var=0 o poner un epsilon
    vars_pos = np.clip(vars_array, 1e-15, None)
    mg = np.exp(np.mean(np.log(vars_pos)))
    return ma / mg


def calcular_ganancias(X_blocks, C_blocks):
    vars_X = np.var(X_blocks, axis=1, ddof=1)
    vars_C = np.var(C_blocks, axis=1, ddof=1)
    GX = ganancia_compaccion(vars_X)
    GC = ganancia_compaccion(vars_C)
    return GX, GC

NameError: name 'X_blocks' is not defined

Filtrado de altas frecuencias

In [7]:
def filtrar_altas_freq(C, M):
    """
    Pone a cero los coeficientes de indice >= M en cada columna.
    C es (N, Nb).
    Retorna la matriz Cfiltrada (mismo shape).
    """
    C_filtr = C.copy()
    C_filtr[M:, :] = 0
    return C_filtr


def practica5_ejemplo(wavfile="v1.wav", N=64, M=32):
    """
    1) Lee v1.wav (mono), expande a multiplo de N.
    2) Forma una matriz (N x Nb) con xE.
    3) Calcula la DCT(64) de todos los bloques (matriz C).
    4) Calcula la ganancia en X y en C (compacción).
    5) Elimina (pone a 0) los coeficientes con indice >= M.
    6) IDCT para recuperar yE.
    7) Recorta ceros y escucha la señal final.
    8) Muestra la relacion señal-a-ruido o el espectro, etc.
    """
    import matplotlib.pyplot as plt

    # 1) Leer y expandir
    xE, fs = leer_y_expandir(wavfile, N)
    L_original = len(
        sf.read(wavfile)[0]
    )  # para saber cuantas muestras tenía sin expandir

    # 2) Matriz X_blocks = (N, Nb)
    X_blocks = xE.reshape(-1, N).T  # (N, Nb)

    # 3) DCT
    C_blocks = dct(X_blocks, axis=0, norm="ortho")  # (N, Nb)

    # 4) Ganancias
    GX, GC = calcular_ganancias(X_blocks, C_blocks)
    print(f"Ganancia en X = {GX:.4f}, Ganancia en C = {GC:.4f}")

    # 5) Filtrar altas freq
    C_filtr = filtrar_altas_freq(C_blocks, M)

    # 6) IDCT
    yE_2d = idct(C_filtr, axis=0, norm="ortho")  # (N, Nb)
    yE = yE_2d.T.reshape(-1)

    # Recortar ceros al final
    y = yE[:L_original]

    # 7) Escuchar (si deseas)
    import sounddevice as sd

    print("Reproduciendo señal filtrada...")
    sd.play(y, fs)
    sd.wait()

    # 8) Métrica SNR:
    #   Tomar x (original) y la parte final y (podríamos re-leer x,
    #   pues xE ya pudo tener ceros finales).
    x, _ = sf.read(wavfile)
    if x.ndim > 1:
        x = x[:, 0]
    # Alineamos longitudes
    L = min(len(x), len(y))
    x = x[:L]
    y = y[:L]

    error = x - y
    energia_signal = np.sum(x**2)
    energia_error = np.sum(error**2)
    if energia_error < 1e-15:
        snr_db = 999.0
    else:
        snr_db = 10 * np.log10(energia_signal / energia_error)

    print(f"SNR(dB) = {snr_db:.2f}")

    # 9) Visualizar
    plt.figure()
    plt.subplot(2, 1, 1)
    plt.plot(x, label="Original")
    plt.title("Señal original")
    plt.subplot(2, 1, 2)
    plt.plot(y, color="r", label="Filtrada")
    plt.title(f"Señal tras filtrar DCT (M={M}) => SNR={snr_db:.2f} dB")
    plt.tight_layout()
    plt.show()